<a href="https://colab.research.google.com/github/DarleneJD/ACOPF/blob/main/13barrasOPF_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [28]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [29]:
!rm -rf ACOPF
!git clone https://github.com/DarleneJD/ACOPF.git

Cloning into 'ACOPF'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 180 (delta 16), reused 0 (delta 0), pack-reused 153 (from 1)
Receiving objects: 100% (180/180), 1.16 MiB | 7.18 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [30]:
import math
from pathlib import Path
import pandas as pd
import pyomo.environ as pyo
from collections import defaultdict
# Solver (HiGHS)
from pyomo.environ import *
import pandas as pd
import numpy as np


# Loading and processing data

In [31]:
base_path = "/content/ACOPF"

In [32]:
def normalize_phases(value):
    """
    Converts inputs like:

    1
    1.0
    '1,2,3'
    '1.0,2.0,3.0'
    '1.2.3'
    into a list of integers [1], [1,2,3]
    """
    if pd.isna(value):
        return []

    s = str(value).strip()

    # unifies separators
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]

    phases = []
    for p in parts:
        try:
            ph = int(float(p))
            if ph in [1, 2, 3]:
                phases.append(ph)
        except ValueError:
            pass

    return sorted(set(phases))




In [33]:
branches_path = f"{base_path}/branches_1.xlsx"
buses_path = f"{base_path}/buses_1.xlsx"

# MT - Medium Voltage
df_branches_mt = pd.read_excel(branches_path, sheet_name="MT")
df_buses_mt = pd.read_excel(buses_path, sheet_name="MT")
df_buses_mt = df_buses_mt[~df_buses_mt["name"].astype(str).str.contains("Source", case=False)]

# BT - Low Voltage
df_branches_bt = pd.read_excel(branches_path, sheet_name="BT")
df_buses_bt = pd.read_excel(buses_path, sheet_name="BT")

# Transformers
df_trafos = pd.read_excel(branches_path, sheet_name="Trafos")
df_trafos.columns = df_trafos.columns.str.strip()

# PV
df_pv = pd.read_excel(buses_path, sheet_name="PV")

#
# NORMALIZATION OF PHASES

df_branches_mt['phase_list'] = df_branches_mt['phase'].apply(normalize_phases)
df_branches_bt['phase_list'] = df_branches_bt['phase'].apply(normalize_phases)
df_trafos['phase_list'] = df_trafos['phases'].apply(normalize_phases)
df_buses_mt['phase_list'] = df_buses_mt['phases'].apply(normalize_phases)
df_buses_bt['phase_list'] = df_buses_bt['phases'].apply(normalize_phases)
df_pv['phase_list'] = df_pv['phases'].apply(normalize_phases)


# MAPPING BARRAS MV

bus_id_mt = dict(zip(df_buses_mt['name'], df_buses_mt['N']))
slack_bus = df_buses_mt.loc[df_buses_mt["tb"] == 3, "N"].iloc[0]
buses_mt = df_buses_mt["N"].astype(int).unique().tolist()


# MAPPING BARRAS LV

bus_id_bt = dict(zip(df_buses_bt['name'], df_buses_bt['N']))
buses_bt = df_buses_bt["N"].astype(int).unique().tolist()


# BRANCHES PREPARATION MV

# Remove branch MT 650->RG60 (It's not a branch, it's an SVR)
df_branches_mt = df_branches_mt[
    ~((df_branches_mt["l"].astype(str) == "650") & (df_branches_mt["k"].astype(str) == "RG60"))
].copy()

# Convert l,k bars to numeric identifier
df_branches_mt['from_bus'] = df_branches_mt['l'].map(bus_id_mt)
df_branches_mt['to_bus'] = df_branches_mt['k'].map(bus_id_mt)

# Remove poorly mapped branches
df_branches_mt = df_branches_mt.dropna(subset=['from_bus', 'to_bus'])

df_branches_mt['from_bus'] = df_branches_mt['from_bus'].astype(int)
df_branches_mt['to_bus'] = df_branches_mt['to_bus'].astype(int)
df_branches_mt['m'] = df_branches_mt['m'].astype(int)
df_branches_mt['Imax'] = df_branches_mt['Imax'].astype(int)

branches_mt = df_branches_mt["m"].unique().tolist()


# BRANCHES PREPARATION LV

df_branches_bt['from_bus'] = df_branches_bt['l'].map(bus_id_bt)
df_branches_bt['to_bus'] = df_branches_bt['k'].map(bus_id_bt)
df_branches_bt = df_branches_bt.dropna(subset=['from_bus', 'to_bus'])

df_branches_bt['from_bus'] = df_branches_bt['from_bus'].astype(int)
df_branches_bt['to_bus'] = df_branches_bt['to_bus'].astype(int)
df_branches_bt['m'] = df_branches_bt['m'].astype(int)

branches_bt = df_branches_bt["m"].unique().tolist()


# SETS BY PHASE

PHASES = [1, 2, 3]

# Branch MV per phase
branch_phase_mt = []
for _, row in df_branches_mt.iterrows():
    m = int(float(row["m"]))
    for ph in row["phase_list"]:
        branch_phase_mt.append((m, ph))
branch_phase_mt = sorted(set(branch_phase_mt))

# Branch LV per phase
branch_phase_bt = []
for _, row in df_branches_bt.iterrows():
    m = int(float(row["m"]))
    for ph in row["phase_list"]:
        branch_phase_bt.append((m, ph))
branch_phase_bt = sorted(set(branch_phase_bt))

# Bus LV per phase
bus_phase_bt = sorted({
    (int(row["N"]), ph)
    for _, row in df_buses_bt.iterrows()
    for ph in row["phase_list"]
})


# Branches MV PARAMETERS
R_mt, X_mt, Imax_mt = {}, {}, {}
from_bus_mt, to_bus_mt, Length_mt, Vbase_mt, Smax_mt = {}, {}, {}, {}, {}

for _, row in df_branches_mt.iterrows():
    m = int(float(row["m"]))
    from_bus_mt[m] = int(row["from_bus"])
    to_bus_mt[m] = int(row["to_bus"])
    Imax_mt[m] = float(row["Imax"])
    Length_mt[m] = float(row["Length_m"])

    # V_base
    fb = from_bus_mt[m]
    Vbase_mt[m] = float(df_buses_mt.loc[df_buses_mt["N"] == fb, "v_nom_kv"].iloc[0])
    Smax_mt[m] = Vbase_mt[m] * Imax_mt[m]  # kVA por fase

    # R,x per phase
    for ph in row["phase_list"]:
        R_mt[(m, ph)] = float(row["R"])
        X_mt[(m, ph)] = float(row["X"])


# Branch LV param
R_bt, X_bt, Imax_bt = {}, {}, {}
from_bus_bt, to_bus_bt, Vbase_bt, Smax_bt = {}, {}, {}, {}

for _, row in df_branches_bt.iterrows():
    m = int(float(row["m"]))
    from_bus_bt[m] = int(row["from_bus"])
    to_bus_bt[m] = int(row["to_bus"])
    Imax_bt[m] = float(row["Imax"])

    # Vbase
    fb = from_bus_bt[m]
    Vbase_bt[m] = float(df_buses_bt.loc[df_buses_bt["N"] == fb, "v_nom_kv"].iloc[0])
    Smax_bt[m] = Vbase_bt[m] * Imax_bt[m]


# Load LV
Pload_bt, Qload_bt = {}, {}

for _, row in df_buses_bt.iterrows():
    b = int(row["N"])
    phs = row["phase_list"]
    if len(phs) == 0:
        continue

    Pd = float(row["P_D"]) if not pd.isna(row["P_D"]) else 0.0
    Qd = float(row["Q_D"]) if not pd.isna(row["Q_D"]) else 0.0

    share = 1.0 / len(phs)

    for ph in phs:
        Pload_bt[(b, ph)] = Pd * share
        Qload_bt[(b, ph)] = Qd * share


# PV

Ppv_bt = {}
Qpv_max = {}

# Initializes with zero for all buses/phases
for (b, ph) in bus_phase_bt:
    Ppv_bt[(b, ph)] = 0.0
    Qpv_max[(b, ph)] = 0.0

for _, row in df_pv.iterrows():
    bus_name = row["Bus"]
    if bus_name not in bus_id_bt:
        continue
    b = int(bus_id_bt[bus_name])

    for ph in row["phase_list"]:
        pcol = f"p_pv_{ph}"
        qcol = f"q_pv_{ph}"
        Ppv_bt[(b, ph)] = float(row[pcol]) if pcol in row and not pd.isna(row[pcol]) else 0.0
        Qpv_max[(b, ph)] = float(row[qcol]) if qcol in row and not pd.isna(row[qcol]) else 0.0


# TRANSFORMERS
# Filter the transformer DataFrame BEFORE creating the sets
df_trafos = df_trafos[df_trafos["trafo_id"] != "Sub"].copy()

# With the filtered DataFrame, create all the sets.
TR = sorted(df_trafos["trafo_id"].unique().tolist())
TR_PH = sorted({
    (tid, ph)
    for _, row in df_trafos.iterrows()
    for tid, phs in [(row["trafo_id"], row["phase_list"])]
    for ph in phs
})

#
# Number of phases per transformer
nph_TR = {row.trafo_id: len(row.phase_list) for _, row in df_trafos.iterrows()}

# Total nominal power (kVA)
Pnom_TR = {row.trafo_id: float(row.kva) for _, row in df_trafos.iterrows()}

# No-load losses (kW)
Pfe_TR = {row.trafo_id: float(row["Perdas Vazio kW"]) for _, row in df_trafos.iterrows()}

# Total losses (kW)
Pt_TR = {row.trafo_id: float(row["Perdas Totais kW"]) for _, row in df_trafos.iterrows()}

# Total copper losses
Pcu_TR = {tid: Pt_TR[tid] - Pfe_TR[tid] for tid in Pnom_TR}

# Copper losses by phase
Pcu_TR_PH = {}
Snom_TR_PH = {}

for _, row in df_trafos.iterrows():
    tid = row.trafo_id
    for ph in row.phase_list:
        Pcu_TR_PH[(tid, ph)] = Pcu_TR[tid] / nph_TR[tid]
        Snom_TR_PH[(tid, ph)] = Pnom_TR[tid] / nph_TR[tid]

# No-load loss per phase
Pfe_TR_PH = {
    (row.trafo_id, ph): Pfe_TR[row.trafo_id] / nph_TR[row.trafo_id]
    for _, row in df_trafos.iterrows()
    for ph in row.phase_list
}

# CLoss coefficients
alpha_tr = {}  # coef perdas P
beta_tr = {}   # coef perdas Q

for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    r_per = float(row["r_per"])/100.0 if ("r_per" in row and not pd.isna(row["r_per"])) else 0.0
    xhl = float(row["XHL"])/100.0 if ("XHL" in row and not pd.isna(row["XHL"])) else 0.0

    for ph in row["phase_list"]:
        alpha_tr[(tid, ph)] = r_per
        beta_tr[(tid, ph)] = xhl

# Rated power of transformers
Snom_TR = {
    row["trafo_id"]: float(row["kva"])
    for _, row in df_trafos.iterrows()
}

# Create a phase dictionary per transformer.
trafo_phases_dict = {row["trafo_id"]: row["phase_list"] for _, row in df_trafos.iterrows()}

# Create virtual buses only for valid transformers
base_tr_bus = 10_000
trafo_MT_bus = {}
trafo_BT_bus = {}

for i, (_, row) in enumerate(df_trafos.iterrows()):
    tid = row["trafo_id"]
    trafo_MT_bus[tid] = base_tr_bus + 2 * i
    trafo_BT_bus[tid] = base_tr_bus + 2 * i + 1

# Mapping between transformers and actual busbars
trafo_to_mt_bus = {}
trafo_to_bt_bus = {}

for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    mt_bus_name = row["mv_bus"]
    bt_bus_name = row["lv_bus"]

    if mt_bus_name in bus_id_mt:
        trafo_to_mt_bus[tid] = bus_id_mt[mt_bus_name]
    else:
        print(f"AVISO: Barra MT '{mt_bus_name}' não encontrada para trafo {tid}")
        # Set a default value
        trafo_to_mt_bus[tid] = buses_mt[0] if buses_mt else -1

    if bt_bus_name in bus_id_bt:
        trafo_to_bt_bus[tid] = bus_id_bt[bt_bus_name]
    else:
        print(f"AVISO: Barra BT '{bt_bus_name}' não encontrada para trafo {tid}")
       # Set a default value
        trafo_to_bt_bus[tid] = buses_bt[0] if buses_bt else -1

# Create sets of bars including virtual ones
all_mt_buses = buses_mt + [trafo_MT_bus[tid] for tid in TR]
all_bt_buses = buses_bt + [trafo_BT_bus[tid] for tid in TR]

# guarantees numeric values ​​(avoids strings like '4,16', etc.)
df_trafos["kv_mv"] = pd.to_numeric(df_trafos["kv_mv"], errors="coerce")
df_trafos["kv_lv"] = pd.to_numeric(df_trafos["kv_lv"], errors="coerce")

# dictionaries by trafo_id
kv_mv_dict = {
    row["trafo_id"]: float(row["kv_mv"])
    for _, row in df_trafos.iterrows()
}
kv_lv_dict = {
    row["trafo_id"]: float(row["kv_lv"])
    for _, row in df_trafos.iterrows()
}


# Modelo

In [34]:
model = pyo.ConcreteModel()

In [35]:
T = list(range(1, 25))  # 0 ... 2880 # Full range with 10-minute steps will be improved later. I'll use 24 hours to try and execute it.
PH = PHASES

In [36]:
# Create artificial IDs for virtual transformer buses
base_tr_bus = 10_000
trafo_MT_bus = {}
trafo_BT_bus = {}

for i, (_, row) in enumerate(df_trafos.iterrows()):
    tid = row["trafo_id"]
    trafo_MT_bus[tid] = base_tr_bus + 2 * i
    trafo_BT_bus[tid] = base_tr_bus + 2 * i + 1

# Mapping between transformers and real busbars
trafo_to_mt_bus = {}
trafo_to_bt_bus = {}

for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    # Map MT bar name to numeric ID
    mt_bus_name = row["mv_bus"]
    bt_bus_name = row["lv_bus"]

    if mt_bus_name in bus_id_mt:
        trafo_to_mt_bus[tid] = bus_id_mt[mt_bus_name]
    if bt_bus_name in bus_id_bt:
        trafo_to_bt_bus[tid] = bus_id_bt[bt_bus_name]

# Create sets of bars including virtual ones
all_mt_buses = buses_mt + [trafo_MT_bus[tid] for tid in TR]
all_bt_buses = buses_bt + [trafo_BT_bus[tid] for tid in TR]

In [37]:
# Model sets
model.BUS_MT = Set(initialize=all_mt_buses, doc="MV buses (including virtual)")
model.BUS_BT = Set(initialize=all_bt_buses, doc="LV buses (including virtual)")
model.T = Set(initialize=T, ordered=True, doc="Time horizon")
model.PH = Set(initialize=PHASES, doc="Phases 1:A, 2:B, 3:C")
model.BR_MT = Set(initialize=branches_mt, doc="MV branches")
model.BR_BT = Set(initialize=branches_bt, doc="LV branches")

In [38]:
# Phase-based sets (including virtual buses)
bus_ph_mt = []
for _, row in df_buses_mt.iterrows():
    b = int(row["N"])
    for ph in row["phase_list"]:
        bus_ph_mt.append((b, ph))

# Add MV virtual buses
for tid in TR:
    phases = trafo_phases_dict[tid]
    for ph in phases:
        bus_ph_mt.append((trafo_MT_bus[tid], ph))

model.BUS_PH_MT = Set(dimen=2, initialize=bus_ph_mt, doc="MV bus-phase pairs")

# LV phase-based sets (including virtual buses)
bus_ph_bt = []
for _, row in df_buses_bt.iterrows():
    b = int(row["N"])
    for ph in row["phase_list"]:
        bus_ph_bt.append((b, ph))

# Add LV virtual buses
for tid in TR:
    phases = trafo_phases_dict[tid]
    for ph in phases:
        bus_ph_bt.append((trafo_BT_bus[tid], ph))

model.BUS_PH_BT = Set(dimen=2, initialize=bus_ph_bt, doc="LV bus-phase pairs")

# Phase-based branches
model.BR_PH_MT = Set(dimen=2, initialize=branch_phase_mt, doc="MV branch-phase pairs")
model.BR_PH_BT = Set(dimen=2, initialize=branch_phase_bt, doc="LV branch-phase pairs")

# Transformers
model.TR = Set(initialize=TR, doc="Transformers")
model.TR_PH = Set(dimen=2, initialize=TR_PH, doc="Transformer-phase pairs")

In [39]:
# Medium Voltage

# MV branch parameters
model.R_MT = Param(model.BR_PH_MT, initialize=lambda model, br, ph: R_mt.get((br, ph), 0.0) / 1000.0, within=NonNegativeReals, doc="Branch resistance in microohms")
model.X_MT = Param(model.BR_PH_MT, initialize=lambda model, br, ph: X_mt.get((br, ph), 0.0) / 1000.0, within=NonNegativeReals, doc="Branch reactance in microohms")
model.Z_sqr_MT = Param(model.BR_PH_MT, initialize=lambda model, br, ph: (R_mt.get((br, ph), 0.0)/1000.0)**2 + (X_mt.get((br, ph), 0.0)/1000.0)**2, doc="Branch impedance")

# Maximum current
Imax_dict_MT = {(br, ph): Imax_mt[br] for (br, ph) in model.BR_PH_MT}
model.I_max_MT = Param(model.BR_PH_MT, initialize=Imax_dict_MT, within=NonNegativeReals, doc="Maximum branch current in A")

# Branch connections
model.from_bus_MT = Param(model.BR_MT, initialize=from_bus_mt)
model.to_bus_MT = Param(model.BR_MT, initialize=to_bus_mt)

# Nominal voltage and limits
model.Vnom_MT = Param(initialize=4.16)  # kV
model.V_min_MT = Param(initialize=0.93 * model.Vnom_MT)
model.V_max_MT = Param(initialize=1.05 * model.Vnom_MT)

# MV loads (zero by default)
model.Pd_MT = Param(model.BUS_PH_MT, model.T, initialize=0, within=Reals)
model.Qd_MT = Param(model.BUS_PH_MT, model.T, initialize=0, within=Reals)

# Slack bus identification
model.Tb = Param(model.BUS_MT, initialize=lambda m, n: 1 if n == slack_bus else 0, doc="Slack bus/Substation")

In [40]:
# Low Voltage

# LV branch parameters
model.R_BT = Param(model.BR_PH_BT, initialize=lambda model, br, ph: R_bt.get((br, ph), 0.0) / 1000.0,  within=NonNegativeReals, doc="Branch resistance in microohms")
model.X_BT = Param(model.BR_PH_BT, initialize=lambda model, br, ph: X_bt.get((br, ph), 0.0) / 1000.0, within=NonNegativeReals, doc="Branch reactance in microohms")
model.Z_sqr_BT = Param(model.BR_PH_BT, initialize=lambda model, br, ph: (R_bt.get((br, ph), 0.0)/1000.0)**2 + (X_bt.get((br, ph), 0.0)/1000.0)**2, doc="Branch impedance")

# LV maximum current
Imax_dict_BT = {(br, ph): Imax_bt[br] for (br, ph) in model.BR_PH_BT}
model.I_max_BT = Param(model.BR_PH_BT, initialize=Imax_dict_BT, within=NonNegativeReals)

# LV branch connections
model.from_bus_BT = Param(model.BR_BT, initialize=from_bus_bt)
model.to_bus_BT = Param(model.BR_BT, initialize=to_bus_bt)

# LV nominal voltage and limits
model.Vnom_BT = Param(initialize=0.480)  # kV
model.V_min_BT = Param(initialize=0.93 * model.Vnom_BT)
model.V_max_BT = Param(initialize=1.05 * model.Vnom_BT)

# LV loads
model.Pd_BT = Param(model.BUS_PH_BT, model.T,
                   initialize=lambda m, b, ph, t: Pload_bt.get((b, ph), 0.0),
                   within=Reals)
model.Qd_BT = Param(model.BUS_PH_BT, model.T,
                   initialize=lambda m, b, ph, t: Qload_bt.get((b, ph), 0.0),
                   within=Reals)

# LV PV
model.Ppv = Param(model.BUS_PH_BT,
                 initialize=lambda m, b, ph: Ppv_bt.get((b, ph), 0.0),
                 within=NonNegativeReals)

def qpv_bounds(m, b, ph, t):
    qmax = Qpv_max.get((b, ph), 0.0)
    return (-qmax, qmax)

# PV variables
model.Qpv = Var(model.BUS_PH_BT, model.T, bounds=qpv_bounds, doc="Reactive power PV")

In [41]:
# Transformers

# Transformer virtual buses (intermediate nodes)
model.tr_mv_bus = Param(model.TR, initialize=trafo_MT_bus, doc="Transformer virtual bus on MV side")
model.tr_lv_bus = Param(model.TR, initialize=trafo_BT_bus, doc="Transformer virtual bus on LV side")

# Real buses connected to transformers
model.trafo_mt_real_bus = Param(model.TR, initialize=trafo_to_mt_bus, doc="Real MV bus connected to transformer")
model.trafo_bt_real_bus = Param(model.TR, initialize=trafo_to_bt_bus, doc="Real LV bus connected to transformer")

# Transformer technical parameters
model.Pnom_TR = Param(model.TR, initialize=Pnom_TR, within=PositiveReals, doc="Nominal Power")
model.Pfe_TR = Param(model.TR, initialize=Pfe_TR, within=NonNegativeReals, doc="No load losses")
model.Pt_TR = Param(model.TR, initialize=Pt_TR, within=NonNegativeReals, doc="Total Losses")
model.Pfe_TR_PH = Param(model.TR_PH, initialize=Pfe_TR_PH, within=NonNegativeReals, doc="No load losses, per trafo and phase")

# Loss coefficients
model.alpha_tr = Param(model.TR_PH, initialize=alpha_tr, default=0.0)
model.beta_tr = Param(model.TR_PH, initialize=beta_tr, default=0.0)

# Rated power per phase
model.Snom_TR_PH = Param(model.TR_PH, initialize=Snom_TR_PH, within=NonNegativeReals, doc="Apparent power")
model.Pcu_nom_TR_PH = Param(model.TR_PH, initialize=Pcu_TR_PH, within=NonNegativeReals, doc="Nominal copper losses, per transformer and per phase")

# Number of phases per transformer
model.nph_TR = Param(model.TR, initialize=nph_TR, within=PositiveIntegers, doc="Number of phases per transformer")
model.cos_phi = Param(initialize=0.92, doc="Minimum required power factor MV-LV" )

# Transformer voltage ratings
model.kv_mv_TR = pyo.Param(model.TR, initialize=kv_mv_dict, within=pyo.PositiveReals)
model.kv_lv_TR = pyo.Param(model.TR, initialize=kv_lv_dict, within=pyo.PositiveReals)

In [42]:
# MV variables
model.I_SQR_MT = Var(model.BR_PH_MT, model.T, domain=NonNegativeReals, doc="Squared current MV")
model.V_SQR_MT = Var(model.BUS_PH_MT, model.T, bounds=(model.V_min_MT ** 2, model.V_max_MT ** 2), domain=NonNegativeReals, initialize=model.Vnom_MT**2, doc="MV squared voltage")

model.P_sub_MT = Var(model.BUS_PH_MT, model.T, domain=NonNegativeReals, initialize=0, doc="Substation active power")
model.Q_sub_MT = Var(model.BUS_PH_MT, model.T, domain=NonNegativeReals, initialize=0, doc="Substation reactive power")

model.P_fluxo_MT = Var(model.BR_PH_MT, model.T, domain=Reals, doc="Active power flow MV")
model.Q_fluxo_MT = Var(model.BR_PH_MT, model.T, domain=Reals, doc="Reactive power flow MV")
model.P_loss_MT = Var(model.BR_PH_MT, model.T, domain=NonNegativeReals, initialize=0, doc="Active power losses MV")

# LV variables
model.I_SQR_BT = Var(model.BR_PH_BT, model.T, domain=NonNegativeReals, doc="Squared current LV")
model.V_SQR_BT = Var(model.BUS_PH_BT, model.T, bounds=(model.V_min_BT ** 2, model.V_max_BT ** 2), domain=NonNegativeReals, initialize=model.Vnom_BT**2, doc="LV squared voltage")

model.P_fluxo_BT = Var(model.BR_PH_BT, model.T, domain=Reals, doc="Active power flow LV")
model.Q_fluxo_BT = Var(model.BR_PH_BT, model.T, domain=Reals,doc="Reactive power flow LV" )
model.P_loss_BT = Var(model.BR_PH_BT, model.T, domain=NonNegativeReals, initialize=0, doc="Active power losses LV")
# Load shedding variable
model.Pshed_BT = Var(model.BUS_PH_BT, model.T, domain=NonNegativeReals, doc="reduction of electrical load when demand exceeds supply")

In [43]:
# Transformer variables
model.Ptr_pos = Var(model.TR_PH, model.T, domain=NonNegativeReals, doc="Positive component of transformer active power flow")
model.Ptr_neg = Var(model.TR_PH, model.T, domain=NonNegativeReals, doc="Negative component of transformer active power flow")
model.Qtr_pos = Var(model.TR_PH, model.T, domain=NonNegativeReals, doc="Positive component of transformer reactive power flow")
model.Qtr_neg = Var(model.TR_PH, model.T, domain=NonNegativeReals, doc="Negative component of transformer reactive power flow")

model.Ptr = Var(model.TR_PH, model.T, domain=Reals, doc="Net active power flow through transformer (MV→LV direction)")
model.Qtr = Var(model.TR_PH, model.T, domain=Reals, doc="Net reactive power flow through transformer (MV→LV direction)")

model.P_loss_CU = Var(model.TR_PH, model.T, domain=NonNegativeReals, doc="Copper losses in transformer per phase")
model.P_loss_TR = pyo.Var(model.TR_PH, model.T, domain=pyo.NonNegativeReals, doc="Total active power losses in transformer per phase")

model.Qloss_tr = Var(model.TR_PH, model.T, domain=NonNegativeReals, doc="Reactive power losses in transformer per phase")

# Variables for connection between real and virtual buses
model.P_fluxo_mt_to_trafo = pyo.Var(model.TR_PH, model.T, domain=pyo.Reals, doc="Active power flow from real MV bus to transformer virtual bus")

model.P_fluxo_trafo_to_bt = pyo.Var(model.TR_PH, model.T, domain=pyo.Reals, doc="Active power flow from transformer virtual bus to real LV bus")

model.Q_fluxo_mt_to_trafo = pyo.Var(model.TR_PH, model.T, domain=pyo.Reals, doc="Reactive power flow from real MV bus to transformer virtual bus")
model.Q_fluxo_trafo_to_bt = pyo.Var(model.TR_PH, model.T, domain=pyo.Reals, doc="Reactive power flow from transformer virtual bus to real LV bus")

model.Ptr_bt = pyo.Var(model.TR_PH, model.T, domain=pyo.Reals, doc="Active power delivered to LV side (after losses)")
model.Qtr_bt = pyo.Var(model.TR_PH, model.T, domain=pyo.Reals, doc="Reactive power delivered to LV side (after losses)")

In [44]:
# Slack bus conditions
for (i, ph) in model.BUS_PH_MT:
    if i == slack_bus:  # MV slack bus
        for t in model.T:
            model.V_SQR_MT[i, ph, t].fix(model.Vnom_MT**2)

# Transformer variable splitting
def ptr_split_rule(model, tid, ph, t):
    """Split transformer active power into positive and negative components"""
    return model.Ptr[tid, ph, t] == model.Ptr_pos[tid, ph, t] - model.Ptr_neg[tid, ph, t]

def qtr_split_rule(model, tid, ph, t):
    """Split transformer reactive power into positive and negative components"""
    return model.Qtr[tid, ph, t] == model.Qtr_pos[tid, ph, t] - model.Qtr_neg[tid, ph, t]

model.PTR_split = pyo.Constraint(model.TR_PH, model.T, rule=ptr_split_rule,
                                 doc="Active power flow decomposition for transformers")
model.QTR_split = pyo.Constraint(model.TR_PH, model.T, rule=qtr_split_rule,
                                 doc="Reactive power flow decomposition for transformers")

# Power rating constraints
def tr_P_rating_rule(m, tid, ph, t):
    """Limit total active power through transformer to rated capacity"""
    return m.Ptr_pos[tid, ph, t] + m.Ptr_neg[tid, ph, t] <= m.Snom_TR_PH[tid, ph] * m.cos_phi

model.TR_P_rating = pyo.Constraint(model.TR_PH, model.T, rule=tr_P_rating_rule,
                                   doc="Transformer active power rating limit")

def tr_Q_rating_rule(m, tid, ph, t):
    """Limit total reactive power through transformer based on power factor"""
    # Calculate reactive capacity based on power factor
    qcap = pyo.sqrt(max(0.0, 1.0 - float(pyo.value(m.cos_phi))**2))
    return m.Qtr_pos[tid, ph, t] + m.Qtr_neg[tid, ph, t] <= m.Snom_TR_PH[tid, ph] * qcap

model.TR_Q_rating = pyo.Constraint(model.TR_PH, model.T, rule=tr_Q_rating_rule,
                                   doc="Transformer reactive power rating limit")

# Transformer losses
def perdas_cobre_trafo_rule(model, tid, ph, t):
    """Calculate copper losses in transformer (quadratic function of load)"""
    Pabs = model.Ptr_pos[tid, ph, t] + model.Ptr_neg[tid, ph, t]
    denom = model.Snom_TR_PH[tid, ph] * model.cos_phi  # kVA*cosphi ~ kW rated per phase

    # denom is Param -> can check in "if" (not a Pyomo expression)
    if pyo.value(denom) == 0:
        return model.P_loss_CU[tid, ph, t] == 0.0

    return model.P_loss_CU[tid, ph, t] == model.Pcu_nom_TR_PH[tid, ph] * (Pabs / denom) ** 2

def perdas_totais_trafo_rule(model, tid, ph, t):
    """Calculate total transformer losses (core + copper)"""
    # Pfe_TR_PH is already per phase
    return model.P_loss_TR[tid, ph, t] == model.Pfe_TR_PH[tid, ph] + model.P_loss_CU[tid, ph, t]

model.perdas_cobre_trafo = pyo.Constraint(model.TR_PH, model.T, rule=perdas_cobre_trafo_rule,
                                          doc="Copper losses in transformers")
model.perdas_totais_trafo = pyo.Constraint(model.TR_PH, model.T, rule=perdas_totais_trafo_rule,
                                           doc="Total active power losses in transformers")

# Transformer reactive losses (heuristic).
#     Warning: beta_tr is a proxy. If this is causing infeasibility,
#     comment this block and fix Qloss_tr=0.
def perdas_reativas_trafo_rule(m, tid, ph, t):
    """Calculate reactive losses in transformer (heuristic approximation)"""
    Pabs = m.Ptr_pos[tid, ph, t] + m.Ptr_neg[tid, ph, t]
    denom = m.Snom_TR_PH[tid, ph] * m.cos_phi
    if pyo.value(denom) == 0:
        return m.Qloss_tr[tid, ph, t] == 0.0
    return m.Qloss_tr[tid, ph, t] == m.beta_tr[tid, ph] * (Pabs / denom) ** 2

model.TR_Qloss = pyo.Constraint(model.TR_PH, model.T, rule=perdas_reativas_trafo_rule,
                                doc="Transformer reactive power losses")

In [45]:
def conservacao_ativa_trafo_rule(m, tid, ph, t):
    """Conservation of active power through transformer: LV power = MV power - losses"""
    return m.Ptr_bt[tid, ph, t] == m.Ptr[tid, ph, t] - m.P_loss_TR[tid, ph, t]

def conservacao_reativa_trafo_rule(m, tid, ph, t):
    """Conservation of reactive power through transformer: LV reactive = MV reactive - reactive losses"""
    return m.Qtr_bt[tid, ph, t] == m.Qtr[tid, ph, t] - m.Qloss_tr[tid, ph, t]

model.TR_conserv_P = pyo.Constraint(model.TR_PH, model.T, rule=conservacao_ativa_trafo_rule,
                                    doc="Active power conservation through transformer (MV to LV)")
model.TR_conserv_Q = pyo.Constraint(model.TR_PH, model.T, rule=conservacao_reativa_trafo_rule,
                                    doc="Reactive power conservation through transformer (MV to LV)")

In [46]:
def tr_vratio_rule(m, tid, ph, t):
    """Transformer voltage ratio constraint: LV voltage = MV voltage × turns ratio"""
    mt_real = m.trafo_mt_real_bus[tid]
    bt_real = m.trafo_bt_real_bus[tid]

    # Skip if either bus-phase pair doesn't exist
    if (mt_real, ph) not in m.BUS_PH_MT or (bt_real, ph) not in m.BUS_PH_BT:
        return pyo.Constraint.Skip

    a = m.kv_lv_TR[tid] / m.kv_mv_TR[tid]  # Voltage ratio (LV/MV) = turns ratio
    return m.V_SQR_BT[bt_real, ph, t] == (a**2) * m.V_SQR_MT[mt_real, ph, t]

model.TR_Vratio = pyo.Constraint(model.TR_PH, model.T, rule=tr_vratio_rule,
                                 doc="Transformer voltage ratio constraint (squared voltages)")

In [47]:
# Define sets of active bus-phase pairs
bus_ph_active_mt = set()
bus_ph_active_bt = set()

# MV – from branches
for (br, ph) in model.BR_PH_MT:
    bus_ph_active_mt.add((model.from_bus_MT[br], ph))
    bus_ph_active_mt.add((model.to_bus_MT[br],   ph))

# MV – from transformers (real side)
for (tid, ph) in model.TR_PH:
    bus_ph_active_mt.add((model.trafo_mt_real_bus[tid], ph))

# MV slack bus (always active)
for ph in model.PH:
    bus_ph_active_mt.add((slack_bus, ph))

# LV – from branches
for (br, ph) in model.BR_PH_BT:
    bus_ph_active_bt.add((model.from_bus_BT[br], ph))
    bus_ph_active_bt.add((model.to_bus_BT[br],   ph))

# LV – from transformers (real side)
for (tid, ph) in model.TR_PH:
    bus_ph_active_bt.add((model.trafo_bt_real_bus[tid], ph))


def pbal_mt_rule(m, bus, ph, t):
    """Active power balance at MV bus: inflow - outflow + slack - to_transformers - load = 0"""

    if (bus, ph) not in bus_ph_active_mt:
        return pyo.Constraint.Skip

    # Power flowing into the bus from branches
    inflow = sum(
        m.P_fluxo_MT[br, ph, t]
        for (br, p) in m.BR_PH_MT
        if p == ph and m.to_bus_MT[br] == bus
    )

    # Power flowing out of the bus to branches
    outflow = sum(
        m.P_fluxo_MT[br, ph, t]
        for (br, p) in m.BR_PH_MT
        if p == ph and m.from_bus_MT[br] == bus
    )

    # Slack bus injection (only at slack bus)
    slack = m.P_sub_MT[bus, ph, t] if m.Tb[bus] == 1 else 0.0
    # Load at bus (if bus exists in this phase)
    load  = m.Pd_MT[bus, ph, t] if (bus, ph) in m.BUS_PH_MT else 0.0

    # Power flowing to transformers connected at this bus
    to_trafos = sum(
        m.P_fluxo_mt_to_trafo[tid, ph, t]
        for tid in m.TR
        if (tid, ph) in m.TR_PH and m.trafo_mt_real_bus[tid] == bus
    )

    return inflow - outflow + slack - to_trafos - load == 0.0

model.MT_Pbal = pyo.Constraint(model.BUS_PH_MT, model.T, rule=pbal_mt_rule,
                               doc="MV active power balance at each bus-phase-time")


def qbal_mt_rule(m, bus, ph, t):
    """Reactive power balance at MV bus: inflow - outflow + slack - to_transformers - load = 0"""

    if (bus, ph) not in bus_ph_active_mt:
        return pyo.Constraint.Skip

    inflow = sum(
        m.Q_fluxo_MT[br, ph, t]
        for (br, p) in m.BR_PH_MT
        if p == ph and m.to_bus_MT[br] == bus
    )

    outflow = sum(
        m.Q_fluxo_MT[br, ph, t]
        for (br, p) in m.BR_PH_MT
        if p == ph and m.from_bus_MT[br] == bus
    )

    slack = m.Q_sub_MT[bus, ph, t] if m.Tb[bus] == 1 else 0.0
    load  = m.Qd_MT[bus, ph, t] if (bus, ph) in m.BUS_PH_MT else 0.0

    to_trafos = sum(
        m.Q_fluxo_mt_to_trafo[tid, ph, t]
        for tid in m.TR
        if (tid, ph) in m.TR_PH and m.trafo_mt_real_bus[tid] == bus
    )

    return inflow - outflow + slack - to_trafos - load == 0.0

model.MT_Qbal = pyo.Constraint(model.BUS_PH_MT, model.T, rule=qbal_mt_rule,
                               doc="MV reactive power balance at each bus-phase-time")


def pbal_bt_rule(m, bus, ph, t):
    """Active power balance at LV bus: inflow - outflow + from_transformers + PV - load = 0"""

    if (bus, ph) not in bus_ph_active_bt:
        return pyo.Constraint.Skip

    inflow = sum(
        m.P_fluxo_BT[br, ph, t]
        for (br, p) in m.BR_PH_BT
        if p == ph and m.to_bus_BT[br] == bus
    )

    outflow = sum(
        m.P_fluxo_BT[br, ph, t]
        for (br, p) in m.BR_PH_BT
        if p == ph and m.from_bus_BT[br] == bus
    )

    load = m.Pd_BT[bus, ph, t]
    pv   = m.Ppv[bus, ph]

    from_trafos = sum(
        m.P_fluxo_trafo_to_bt[tid, ph, t]
        for tid in m.TR
        if (tid, ph) in m.TR_PH and m.trafo_bt_real_bus[tid] == bus
    )

    return inflow - outflow + from_trafos + pv - load == 0.0

model.BT_Pbal = pyo.Constraint(model.BUS_PH_BT, model.T, rule=pbal_bt_rule,
                               doc="LV active power balance at each bus-phase-time")


def qbal_bt_rule(m, bus, ph, t):
    """Reactive power balance at LV bus: inflow - outflow + from_transformers + PV - load = 0"""

    if (bus, ph) not in bus_ph_active_bt:
        return pyo.Constraint.Skip

    inflow = sum(
        m.Q_fluxo_BT[br, ph, t]
        for (br, p) in m.BR_PH_BT
        if p == ph and m.to_bus_BT[br] == bus
    )

    outflow = sum(
        m.Q_fluxo_BT[br, ph, t]
        for (br, p) in m.BR_PH_BT
        if p == ph and m.from_bus_BT[br] == bus
    )

    load = m.Qd_BT[bus, ph, t]
    pv   = m.Qpv[bus, ph, t]

    from_trafos = sum(
        m.Q_fluxo_trafo_to_bt[tid, ph, t]
        for tid in m.TR
        if (tid, ph) in m.TR_PH and m.trafo_bt_real_bus[tid] == bus
    )

    return inflow - outflow + from_trafos + pv - load == 0.0

model.BT_Qbal = pyo.Constraint(model.BUS_PH_BT, model.T, rule=qbal_bt_rule,
                               doc="LV reactive power balance at each bus-phase-time")

In [48]:
# Voltage drop equations (DistFlow formulation)

def mt_vdrop_rule(model, br, ph, t):
    """Voltage drop constraint for MV branch: V_j² = V_i² - 2(RP+XQ) + Z²I²"""
    i = model.from_bus_MT[br]
    j = model.to_bus_MT[br]

    # Skip if either bus-phase pair doesn't exist
    if (i, ph) not in model.BUS_PH_MT or (j, ph) not in model.BUS_PH_MT:
        return Constraint.Skip

    return model.V_SQR_MT[j, ph, t] == (
        model.V_SQR_MT[i, ph, t]
        - 2 * (model.R_MT[br, ph] * model.P_fluxo_MT[br, ph, t] +
               model.X_MT[br, ph] * model.Q_fluxo_MT[br, ph, t])
        + model.Z_sqr_MT[br, ph] * model.I_SQR_MT[br, ph, t]
    )

def bt_vdrop_rule(model, br, ph, t):
    """Voltage drop constraint for LV branch: V_j² = V_i² - 2(RP+XQ) + Z²I²"""
    i = model.from_bus_BT[br]
    j = model.to_bus_BT[br]

    # Skip if either bus-phase pair doesn't exist
    if (i, ph) not in model.BUS_PH_BT or (j, ph) not in model.BUS_PH_BT:
        return Constraint.Skip

    return model.V_SQR_BT[j, ph, t] == (
        model.V_SQR_BT[i, ph, t]
        - 2 * (model.R_BT[br, ph] * model.P_fluxo_BT[br, ph, t] +
               model.X_BT[br, ph] * model.Q_fluxo_BT[br, ph, t])
        + model.Z_sqr_BT[br, ph] * model.I_SQR_BT[br, ph, t]
    )

model.MT_Vdrop = Constraint(model.BR_PH_MT, model.T, rule=mt_vdrop_rule,
                            doc="MV branch voltage drop (DistFlow formulation)")
model.BT_Vdrop = Constraint(model.BR_PH_BT, model.T, rule=bt_vdrop_rule,
                            doc="LV branch voltage drop (DistFlow formulation)")

In [49]:
# Power flow - current relationship (apparent power limit)

def mt_flow_rule(model, br, ph, t):
    """Apparent power constraint for MV branch: P² + Q² ≤ V² × I²"""
    i = model.from_bus_MT[br]

    # Skip if the bus-phase pair doesn't exist
    if (i, ph) not in model.BUS_PH_MT:
        return Constraint.Skip

    return (model.P_fluxo_MT[br, ph, t]**2 + model.Q_fluxo_MT[br, ph, t]**2 <=
            model.V_SQR_MT[i, ph, t] * model.I_SQR_MT[br, ph, t])

def bt_flow_rule(model, br, ph, t):
    """Apparent power constraint for LV branch: P² + Q² ≤ V² × I²"""
    i = model.from_bus_BT[br]

    # Skip if the bus-phase pair doesn't exist
    if (i, ph) not in model.BUS_PH_BT:
        return Constraint.Skip

    return (model.P_fluxo_BT[br, ph, t]**2 + model.Q_fluxo_BT[br, ph, t]**2 <=
            model.V_SQR_BT[i, ph, t] * model.I_SQR_BT[br, ph, t])

model.fluxo_potencia_mt = Constraint(model.BR_PH_MT, model.T, rule=mt_flow_rule,
                                     doc="MV branch apparent power limit (P²+Q² ≤ V²I²)")
model.fluxo_potencia_bt = Constraint(model.BR_PH_BT, model.T, rule=bt_flow_rule,
                                     doc="LV branch apparent power limit (P²+Q² ≤ V²I²)")

In [50]:
# Current limits

def i_max_mt_rule(model, br, ph, t):
    """Current limit constraint for MV branch: I² ≤ I_max²"""
    return model.I_SQR_MT[br, ph, t] <= model.I_max_MT[br, ph]**2

def i_max_bt_rule(model, br, ph, t):
    """Current limit constraint for LV branch: I² ≤ I_max²"""
    return model.I_SQR_BT[br, ph, t] <= model.I_max_BT[br, ph]**2

model.i_max_mt = Constraint(model.BR_PH_MT, model.T, rule=i_max_mt_rule,
                            doc="MV branch current limit (thermal rating)")
model.i_max_bt = Constraint(model.BR_PH_BT, model.T, rule=i_max_bt_rule,
                            doc="LV branch current limit (thermal rating)")

In [51]:
# Objective function
def objective_rule(model):
    """Objective: Minimize total system losses, substation energy cost, and load shedding penalty"""

    # MV line losses
    perdas_mt = sum(model.P_loss_MT[br, ph, t]
                   for (br, ph) in model.BR_PH_MT
                   for t in model.T)

    # LV line losses
    perdas_bt = sum(model.P_loss_BT[br, ph, t]
                   for (br, ph) in model.BR_PH_BT
                   for t in model.T)

    # Transformer losses
    perdas_tr = sum(model.P_loss_TR[tid, ph, t]
                   for (tid, ph) in model.TR_PH
                   for t in model.T)

    # Substation energy cost (purchased from grid)
    custo_energia = sum(model.P_sub_MT[i, ph, t]
                       for (i, ph) in model.BUS_PH_MT
                       for t in model.T)

    # Load shedding penalty (high penalty to discourage load shedding)
    penalty_shed = 1e4 * sum(model.Pshed_BT[b, ph, t]
                         for (b, ph) in model.BUS_PH_BT
                         for t in model.T)

    # Minimize: total losses + 0.1 × energy cost + load shedding penalty
    return perdas_mt + perdas_bt + perdas_tr + 0.1 * custo_energia + penalty_shed

model.objective = Objective(rule=objective_rule, sense=minimize,
                            doc="Minimize total system losses, energy cost, and load shedding")

In [52]:
# Solver configuration and execution
opt = SolverFactory('ipopt')
results = opt.solve(model, tee=True, options={'max_iter': 10000})
print(results)

Ipopt 3.12.13: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    32016
Number of nonzeros in inequality constraint Jacobian.:    15384
Number of nonzeros in Lagrangian Hessian.............:     9504

Total number of variables............................:    28224
                     variables with only lower bounds:    13536
                variables with lower and upper bounds:     5280
                     variables with only upper bound

# Regulador

In [53]:
# Mapeamento explícito regulador → fase
reg_phase_map = {
    "Reg1": 1,
    "Reg2": 2,
    "Reg3": 3,
}

reg_phase = [(rid, ph) for rid, ph in reg_phase_map.items()]

model.REG_PH = Set(dimen=2, initialize=reg_phase)



In [54]:
reg_mv_bus = {row["trafo_id"]: int(bus_id_map[row["mv_bus"]]) for _, row in df_reg.iterrows()}
reg_lv_bus = {row["trafo_id"]: int(bus_id_map[row["lv_bus"]]) for _, row in df_reg.iterrows()}

vreg = {}
band = {}
step = {}
tap_min = {}
tap_max = {}

NTAPS = 16  # <<< DEFINIÇÃO CORRETA (engenharia)

for _, row in df_reg.iterrows():
    rid = row["trafo_id"]
    if rid not in reg_phase_map:
        continue

    ph = reg_phase_map[rid]

    vreg[(rid,ph)] = row["vreg_V"] / (row["kv_mv"] * 1000)
    band[(rid,ph)] = row["band_V"] / (row["kv_mv"] * 1000)

    # Step (% → pu)
    step[(rid,ph)] = float(row["Step"]) / 100.0

    tap_min[(rid,ph)] = -NTAPS
    tap_max[(rid,ph)] =  NTAPS


NameError: name 'df_reg' is not defined

In [ ]:
model.tap = Var(model.REG_PH, model.T, domain=Integers)


In [ ]:
def tap_bounds(m, rid, ph, t):
    return (tap_min[(rid,ph)], tap_max[(rid,ph)])

model.tap = Var(model.REG_PH, model.T, domain=Integers, bounds=tap_bounds)


In [ ]:
# NÃO aplicar regulador se j == mv_bus do trafo
def reg_voltage(mm, rid, ph, t):
    i = reg_mv_bus[rid]
    j = reg_lv_bus[rid]
    if j in trafo_mv_bus.values():
        return Constraint.Skip
    return mm.v_MT[j,ph,t] == mm.v_MT[i,ph,t] + 2*step[(rid,ph)]*mm.tap[rid,ph,t]


model.RegVoltage = Constraint(model.REG_PH, model.T, rule=reg_voltage_rule)


In [ ]:
model.over_v = Var(model.REG_PH, model.T, domain=Binary)
model.under_v = Var(model.REG_PH, model.T, domain=Binary)


In [ ]:
M = 10.0

def over_voltage_rule(m, rid, ph, t):
    j = reg_lv_bus[rid]
    return m.v[j,ph] - (vreg[(rid,ph)] + band[(rid,ph)]/2) <= M * m.over_v[rid,ph,t]

def under_voltage_rule(m, rid, ph, t):
    j = reg_lv_bus[rid]
    return (vreg[(rid,ph)] - band[(rid,ph)]/2) - m.v[j,ph] <= M * m.under_v[rid,ph,t]

model.OverV = Constraint(model.REG_PH, model.T, rule=over_voltage_rule)
model.UnderV = Constraint(model.REG_PH, model.T, rule=under_voltage_rule)


In [ ]:
t0 = T_list[0]
for (rid,ph) in m.REG_PH:
    m.tap[rid,ph,t0].fix(0)
    m.tap_up[rid,ph,t0].fix(0)
    m.tap_down[rid,ph,t0].fix(0)



def tap_evolution_rule(m, rid, ph, t):
    if t == 0:
        return Constraint.Skip
    return (
        m.tap[rid,ph,t]
        == m.tap[rid,ph,t-1]
        + m.tap_up[rid,ph,t]
        - m.tap_down[rid,ph,t]
    )

model.TapEvolution = Constraint(model.REG_PH, model.T, rule=tap_evolution_rule)


In [ ]:
def reg_energy_conservation(m, rid, ph, t):
    # Regulador não gera potência
    return m.Preg[rid,ph,t] == 0
m.REG_NoGenP = pyo.Constraint(m.REG_PH, m.T, rule=reg_energy_conservation)


In [ ]:
def tap_up_rule(m, rid, ph, t):
    return m.tap_up[rid,ph,t] <= m.under_v[rid,ph,t]

def tap_down_rule(m, rid, ph, t):
    return m.tap_down[rid,ph,t] <= m.over_v[rid,ph,t]

model.TapUpRule = Constraint(model.REG_PH, model.T, rule=tap_up_rule)
model.TapDownRule = Constraint(model.REG_PH, model.T, rule=tap_down_rule)
